In [2]:
%pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [4]:

import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.optimize import root_scalar

# ------------------------- Black-Scholes Functions -------------------------
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    if T <= 0 or sigma <= 0:
        return 0, 0, 0, 0, 0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    delta = norm.cdf(d1) if option_type == 'call' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) -
             r * K * np.exp(-r * T) * norm.cdf(d2)) / 365 if option_type == 'call' else             (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) +
             r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100 if option_type == 'call' else           -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    return delta, gamma, vega, theta, rho

def calculate_pnl(S, K, T, r, sigma, purchase_price, option_type='call'):
    return black_scholes_price(S, K, T, r, sigma, option_type) - purchase_price

def generate_price_grid(S_range, vol_range, K, T, r, option_type='call'):
    return pd.DataFrame(
        [[black_scholes_price(S, K, T, r, sigma, option_type) for S in S_range] for sigma in vol_range],
        index=[f"{v:.2f}" for v in vol_range], columns=[f"{s:.2f}" for s in S_range]
    )

def generate_pnl_grid(S_range, vol_range, K, T, r, purchase_price, option_type='call'):
    return pd.DataFrame(
        [[black_scholes_price(S, K, T, r, sigma, option_type) - purchase_price for S in S_range] for sigma in vol_range],
        index=[f"{v:.2f}" for v in vol_range], columns=[f"{s:.2f}" for s in S_range]
    )

def implied_volatility(market_price, S, K, T, r, option_type='call'):
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - market_price
    try:
        result = root_scalar(objective, bracket=[1e-5, 3], method='brentq')
        return result.root if result.converged else None
    except:
        return None

# ------------------------- Streamlit Interface -------------------------
st.set_page_config(page_title="Black-Scholes Dashboard", layout="wide")
st.markdown("<h1 style='text-align: center;'> Black-Scholes Option Pricing Dashboard - Auro</h1>", unsafe_allow_html=True)

# Sidebar Inputs
st.sidebar.markdown("## ⚙️ Model Inputs")
S = st.sidebar.number_input("**Stock Price (S)**", value=100.0)
K = st.sidebar.number_input("**Strike Price (K)**", value=100.0)
T = st.sidebar.number_input("**Time to Maturity (T, years)**", value=1.0, min_value=0.001)
sigma = st.sidebar.number_input("**Volatility (σ)**", value=0.2, min_value=0.001)
r = st.sidebar.number_input("**Risk-Free Rate (r)**", value=0.05)
option_type = st.sidebar.radio("**Option Type**", ["call", "put"])
purchase_price = st.sidebar.number_input("**Purchase Price**", value=8.00)
market_price = st.sidebar.number_input("**Observed Market Price (for IV)**", value=10.00)

# Test Case Example
with st.expander(" **Test Case Example: AAPL Option (Aug 2025)**", expanded=False):
    st.markdown("""
    - Stock Price = `196.25`
    - Strike Price = `195.00`
    - Time to Maturity = `0.0164` years
    - Volatility = `0.22`
    - Risk-Free Rate = `0.05`
    - Purchase Price = `2.00`
    - Market Price = `2.25`
    """)

# Section A: Pricing & Greeks
st.divider()
st.markdown("###  **Option Pricing & Greeks**")
if T <= 0 or sigma <= 0:
    st.error("Time to maturity (T) and volatility (σ) must be positive.")
else:
    option_price = black_scholes_price(S, K, T, r, sigma, option_type)
    delta, gamma, vega, theta, rho = calculate_greeks(S, K, T, r, sigma, option_type)
    col_price, col_greeks = st.columns([1.2, 1.5])
    with col_price:
        st.success(f"**Option Price:** ${option_price:.2f}")
    with col_greeks:
        with st.expander(" **Greeks (Δ, Γ, Vega, Theta, Rho)**", expanded=False):
            st.markdown(f"**Delta (Δ):** {delta:.4f}")
            st.markdown(f"**Gamma (Γ):** {gamma:.4f}")
            st.markdown(f"**Vega:** {vega:.4f}")
            st.markdown(f"**Theta:** {theta:.4f}")
            st.markdown(f"**Rho:** {rho:.4f}")

# Section B: P&L
st.divider()
st.markdown("###  **Profit or Loss (P&L)**")
pnl = calculate_pnl(S, K, T, r, sigma, purchase_price, option_type)
if pnl >= 0:
    st.success(f"**Profit:** ${pnl:.2f}")
else:
    st.error(f"**Loss:** ${pnl:.2f}")

# Section C: Sensitivity Heatmap
st.divider()
st.markdown("### 🔍 **Shock Analysis (Sensitivity Heatmap)**")
enable_analysis = st.checkbox("Enable Sensitivity Analysis", value=False)
if enable_analysis:
    col1, col2 = st.columns(2)
    with col1:
        S_min = st.number_input("Min Stock Price", value=80.0)
        S_max = st.number_input("Max Stock Price", value=120.0)
    with col2:
        vol_min = st.number_input("Min Volatility", value=0.1)
        vol_max = st.number_input("Max Volatility", value=0.5)
    steps = st.slider("Resolution (Higher = More Detail)", min_value=5, max_value=50, value=20)
    heatmap_type = st.radio("Heatmap Type", ["Option Price", "P&L"])
    if st.button("Generate Heatmap"):
        S_range = np.linspace(S_min, S_max, steps)
        vol_range = np.linspace(vol_min, vol_max, steps)
        df = generate_price_grid(S_range, vol_range, K, T, r, option_type) if heatmap_type == "Option Price" else              generate_pnl_grid(S_range, vol_range, K, T, r, purchase_price, option_type)
        title = f"{heatmap_type} Heatmap"
        center_val = 0 if heatmap_type == "P&L" else None
        fig, ax = plt.subplots(figsize=(9, 5))
        sns.heatmap(df, cmap="coolwarm" if heatmap_type == "P&L" else "YlGnBu",
                    center=center_val, annot=False, fmt=".2f", ax=ax)
        ax.set_title(title, fontsize=14)
        st.pyplot(fig)

# Section D: Implied Volatility
st.divider()
st.markdown("###  **Implied Volatility (IV)**")
iv = implied_volatility(market_price, S, K, T, r, option_type)
if iv is not None:
    st.info(f"**Implied Volatility:** {iv:.4f} ({iv*100:.2f}%)")
else:
    st.warning("Could not calculate IV. Adjust inputs.")


2025-08-05 19:32:12.505 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.508 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.509 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.511 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.512 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 19:32:12.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar